In [7]:
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime, timedelta
import random 

In [8]:
stock = 'SRFM'
t = 3

In [9]:
def plot_stock_candlestick(symbol, months=3):
    """
    Plot candlestick chart for any stock
    
    Args:
        symbol: Stock ticker symbol (e.g., 'AAPL', 'TSLA')
        months: Number of months of historical data (default: 3)
    """
    # Get data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=months*30)
    
    df = yf.download(symbol, start=start_date, end=end_date, progress=False)
    ticker = yf.Ticker(symbol)
    company_name = ticker.info.get('longName', symbol)
    
    # Process data
    df.reset_index(inplace=True)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    
    # Create figure
    fig = go.Figure(data=[go.Candlestick(
        x=df['Date'],
        open=df['Open'].squeeze(),
        high=df['High'].squeeze(),
        low=df['Low'].squeeze(),
        close=df['Close'].squeeze()
    )])
    
    # Calculate stats
    current_price = df['Close'].iloc[-1]
    first_price = df['Close'].iloc[0]
    price_change = current_price - first_price
    percent_change = (price_change / first_price) * 100
    
    # Update layout
    fig.update_layout(
        title=f'{company_name} ({symbol}) - Last {months} Months<br>' +
              f'<sub>Current: ${current_price:.2f} | Change: ${price_change:.2f} ({percent_change:+.2f}%)</sub>',
        yaxis_title=f'{symbol} Stock Price (USD)',
        xaxis_title='Date',
        hovermode='x unified'
    )
    
    return fig

# ========== USE THE FUNCTION ==========
# Change the stock symbol and months here:
fig = plot_stock_candlestick(stock, months=t)
fig.show()

# You can easily plot multiple stocks:
# plot_stock_candlestick('TSLA', months=6).show()
# plot_stock_candlestick('MSFT', months=1).show()
# plot_stock_candlestick('GOOGL', months=12).show()

/var/folders/0h/ldts3ql12nldxhdrs9tccd_00000gn/T/ipykernel_67770/1616883206.py:13: FutureWarning:

YF.download() has changed argument auto_adjust default to True



In [10]:
# Get the same data for analysis
symbol = stock
months = t
end_date = datetime.now()
start_date = end_date - timedelta(days=months*30)

# Download data
df = yf.download(symbol, start=start_date, end=end_date, progress=False)

# Process data
df.reset_index(inplace=True)
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

# Convert column names to lowercase for the engulfing pattern function
df.columns = df.columns.str.lower()

print(f"Downloaded {len(df)} rows of {symbol} data")
df.tail()


Downloaded 63 rows of SRFM data


/var/folders/0h/ldts3ql12nldxhdrs9tccd_00000gn/T/ipykernel_67770/2252787733.py:8: FutureWarning:

YF.download() has changed argument auto_adjust default to True



Price,date,close,high,low,open,volume
58,2025-11-17,2.51,2.600,2.45,2.60,3069600
59,2025-11-18,2.38,2.555,2.36,2.42,2462700
60,2025-11-19,2.22,2.390,2.17,2.37,3073400
61,2025-11-20,2.06,2.530,2.06,2.35,4389500
62,2025-11-21,2.07,2.080,1.94,2.08,2760300


In [11]:
import numpy as np
from plotly.subplots import make_subplots

def calculate_rsi(data, periods=14):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_momentum(data, periods=10):
    """Calculate Price Momentum"""
    return data.diff(periods)

def identify_momentum_trend(df):
    """
    Identify momentum trends in the data
    Returns df with momentum indicators and trend signals
    """
    # Calculate momentum indicators
    df['rsi'] = calculate_rsi(df['close'], periods=14)
    df['momentum'] = calculate_momentum(df['close'], periods=10)
    df['sma_20'] = df['close'].rolling(window=20).mean()
    df['sma_50'] = df['close'].rolling(window=50).mean()
    
    # Price change and momentum signals
    df['price_change'] = df['close'].pct_change() * 100
    df['momentum_strength'] = df['momentum'] / df['close'] * 100
    
    # Identify trend signals
    df['bullish_momentum'] = (
        (df['rsi'] > 50) & 
        (df['close'] > df['sma_20']) & 
        (df['sma_20'] > df['sma_50']) &
        (df['momentum'] > 0)
    )
    
    df['bearish_momentum'] = (
        (df['rsi'] < 50) & 
        (df['close'] < df['sma_20']) & 
        (df['sma_20'] < df['sma_50']) &
        (df['momentum'] < 0)
    )
    
    # Strong momentum signals
    df['strong_bullish'] = (df['rsi'] > 70) | (df['momentum_strength'] > 5)
    df['strong_bearish'] = (df['rsi'] < 30) | (df['momentum_strength'] < -5)
    
    return df

# Apply momentum analysis
df = identify_momentum_trend(df)

# Display key statistics
print(f"\n{'='*60}")
print(f"MOMENTUM ANALYSIS for {stock.upper()}")
print(f"{'='*60}")
print(f"\nCurrent Statistics (Latest Trading Day):")
print(f"  Close Price:        ${df['close'].iloc[-1]:.2f}")
print(f"  RSI (14):           {df['rsi'].iloc[-1]:.2f}")
print(f"  Momentum (10-day):  ${df['momentum'].iloc[-1]:.2f}")
print(f"  Momentum Strength:  {df['momentum_strength'].iloc[-1]:.2f}%")
print(f"  20-day SMA:         ${df['sma_20'].iloc[-1]:.2f}")
print(f"  50-day SMA:         ${df['sma_50'].iloc[-1]:.2f}")

# Count trend signals in recent data (last 30 days)
recent_data = df.tail(30)
bullish_days = recent_data['bullish_momentum'].sum()
bearish_days = recent_data['bearish_momentum'].sum()
strong_bull_days = recent_data['strong_bullish'].sum()
strong_bear_days = recent_data['strong_bearish'].sum()

print(f"\n30-Day Trend Summary:")
print(f"  Bullish Momentum Days:  {bullish_days} ({bullish_days/30*100:.1f}%)")
print(f"  Bearish Momentum Days:  {bearish_days} ({bearish_days/30*100:.1f}%)")
print(f"  Strong Bullish Signals: {strong_bull_days}")
print(f"  Strong Bearish Signals: {strong_bear_days}")

# Current trend determination
current_trend = "BULLISH 📈" if df['bullish_momentum'].iloc[-1] else \
                "BEARISH 📉" if df['bearish_momentum'].iloc[-1] else \
                "NEUTRAL ➡️"
print(f"\nCurrent Momentum Trend: {current_trend}")

if df['strong_bullish'].iloc[-1]:
    print("⚠️  WARNING: Overbought conditions detected (RSI > 70 or strong upward momentum)")
elif df['strong_bearish'].iloc[-1]:
    print("⚠️  WARNING: Oversold conditions detected (RSI < 30 or strong downward momentum)")

print(f"{'='*60}\n")

# Display recent data with momentum indicators
print("\nRecent Trading Days with Momentum Indicators:")
display_cols = ['date', 'close', 'rsi', 'momentum', 'momentum_strength']
print(df[display_cols].tail(10).to_string(index=False))



MOMENTUM ANALYSIS for SRFM

Current Statistics (Latest Trading Day):
  Close Price:        $2.07
  RSI (14):           5.05
  Momentum (10-day):  $-1.25
  Momentum Strength:  -60.39%
  20-day SMA:         $3.16
  50-day SMA:         $4.05

30-Day Trend Summary:
  Bullish Momentum Days:  0 (0.0%)
  Bearish Momentum Days:  12 (40.0%)
  Strong Bullish Signals: 3
  Strong Bearish Signals: 25

Current Momentum Trend: BEARISH 📉
⚠️  WARNING: Oversold conditions detected (RSI < 30 or strong downward momentum)


Recent Trading Days with Momentum Indicators:
      date  close       rsi  momentum  momentum_strength
2025-11-10   3.14 16.894981     -1.08         -34.394893
2025-11-11   3.04 18.686870     -1.00         -32.894737
2025-11-12   2.94 17.874401     -1.02         -34.693876
2025-11-13   2.69 12.556052     -1.18         -43.866164
2025-11-14   2.63 13.023257     -1.43         -54.372615
2025-11-17   2.51 13.397128     -1.34         -53.386451
2025-11-18   2.38 13.084112     -1.06        

In [12]:
def plot_momentum_candlestick(df, symbol):
    """
    Plot candlestick chart with momentum indicators
    """
    # Create subplots: candlestick on top, RSI and momentum below
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.02,
        row_heights=[0.55, 0.22, 0.23]
    )
    
    # Add candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=df['date'],
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            name='Price'
        ),
        row=1, col=1
    )
    
    # Add moving averages
    fig.add_trace(
        go.Scatter(
            x=df['date'], 
            y=df['sma_20'],
            name='SMA 20',
            line=dict(color='orange', width=1.5)
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df['date'], 
            y=df['sma_50'],
            name='SMA 50',
            line=dict(color='blue', width=1.5)
        ),
        row=1, col=1
    )
    
    # Add bullish/bearish momentum markers
    bullish_points = df[df['bullish_momentum']]
    bearish_points = df[df['bearish_momentum']]
    
    if not bullish_points.empty:
        fig.add_trace(
            go.Scatter(
                x=bullish_points['date'],
                y=bullish_points['high'] * 1.02,
                mode='markers',
                marker=dict(symbol='triangle-up', size=8, color='green'),
                name='Bullish Momentum',
                showlegend=True
            ),
            row=1, col=1
        )
    
    if not bearish_points.empty:
        fig.add_trace(
            go.Scatter(
                x=bearish_points['date'],
                y=bearish_points['low'] * 0.98,
                mode='markers',
                marker=dict(symbol='triangle-down', size=8, color='red'),
                name='Bearish Momentum',
                showlegend=True
            ),
            row=1, col=1
        )
    
    # Add RSI
    fig.add_trace(
        go.Scatter(
            x=df['date'],
            y=df['rsi'],
            name='RSI',
            line=dict(color='purple', width=2)
        ),
        row=2, col=1
    )
    
    # Add RSI overbought/oversold lines
    fig.add_hline(y=70, line_dash="dash", line_color="red", opacity=0.5, row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", opacity=0.5, row=2, col=1)
    fig.add_hline(y=50, line_dash="dot", line_color="gray", opacity=0.3, row=2, col=1)
    
    # Add Momentum
    fig.add_trace(
        go.Scatter(
            x=df['date'],
            y=df['momentum'],
            name='Momentum',
            line=dict(color='teal', width=2),
            fill='tozeroy'
        ),
        row=3, col=1
    )
    
    # Add zero line for momentum
    fig.add_hline(y=0, line_dash="solid", line_color="gray", opacity=0.5, row=3, col=1)
    
    # Update layout
    fig.update_layout(
        height=950,
        title=f'<b>{symbol.upper()} Momentum Analysis</b><br>' +
              f'<sub>RSI: {df["rsi"].iloc[-1]:.2f} | ' +
              f'Momentum: ${df["momentum"].iloc[-1]:.2f} | ' +
              f'Trend: {"Bullish 📈" if df["bullish_momentum"].iloc[-1] else "Bearish 📉" if df["bearish_momentum"].iloc[-1] else "Neutral ➡️"}</sub>',
        hovermode='x unified',
        showlegend=True,
        xaxis3_title='Date',
        margin=dict(t=100, b=80),
        xaxis_rangeslider_visible=False,
        xaxis2_rangeslider_visible=False,
        xaxis3_rangeslider_visible=False
    )
    
    # Update y-axes with descriptive titles
    fig.update_yaxes(title_text="<b>Price (USD)</b><br>w/ SMA 20 & 50", row=1, col=1)
    fig.update_yaxes(title_text="<b>RSI (14)</b>", row=2, col=1, range=[0, 100])
    fig.update_yaxes(title_text="<b>Momentum (10-day)</b>", row=3, col=1)
    
    return fig

# Create and display the visualization
momentum_fig = plot_momentum_candlestick(df, stock)
momentum_fig.show()
